# 拉普拉斯方法定轨道六根数

## 1. 已知三次观测的观测坐标，定出某一时刻的位置矢量和速度矢量
按照笔记的方法分别推出各个量

注意时间单位的选取!

In [12]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime
import time

from DDS_Laplace import *

import DDS_CMpack as DDS

%matplotlib inline

## PART 1 星表的format

In [13]:
# astronomy constants
R_earth = 6371
au = 149597870
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 


In [14]:
# give 3 observed position
# position_fram = pd.DataFrame({'alpha':\
#                         np.array([142.935,157.274166667,171.817916667,183.134166667,\
#                                   208.640416667,219.102083333]),\
#                        'delta':\
#                        np.array([8.521111111,-2.395,-14.508333333,-23.56333333,-40.020277777,-44.998611111]),\
#                        'UT1_hours':np.array([21.575128333,21.60311055,21.631419722,\
#                                              21.654491388,21.714094166,21.742875555])})

position_fram = pd.read_csv('Echo1st.csv')

position_fram

,UT1_hours,alpha,delta,year,month,days
0,21.575128,142.935000,8.521111,1965,1,14
1,21.603111,157.274167,-2.395000,1965,1,14
2,21.631420,171.817917,-14.508333,1965,1,14
3,21.654491,183.134167,-23.563333,1965,1,14
4,21.714094,208.640417,-40.020278,1965,1,14
5,21.742876,219.102083,-44.998611,1965,1,14


In [15]:
# 计算观测数据的儒略日时间和恒星时
year = 1965; month = 1; days = 14;
UT1_hours = np.array([21.575128333,21.60311055,21.631419722,21.654491388,21.714094166,21.742875555])
jd_t = []
SG_t = []
for i in range(len(UT1_hours)):
    SG_s,jd_s = DDS.UTC2SG(year,month,days,UT1_hours[i])
    SG_t.append(SG_s); jd_t.append(jd_s);
    
SG_array = np.array(SG_t); 
SG_degree = SG_array%(3600*24)/(3600*24)*360

jd_array = np.array(jd_t)
delta_t = (UT1_hours - UT1_hours[0])*3600/time_unit
delta_t_SI = delta_t * time_unit
position_fram['deltaT'] = delta_t

delta_t

array([0.        , 0.12485688, 0.25117263, 0.35411859, 0.62006667,
       0.74848946])

In [16]:
# give the station's lam and phi
lam = 118.82091666
phi = 31.893611111


In [17]:
# 需要根据alpha和delta求出所需的测站到地心的RA,DEC
station_earth = []
for i in range(len(position_fram)):
    # give the station's RA and DEC then convert to station_earth
    RA_station = SG_degree[i] + lam
    DEC_station = phi
    station_earth.append(DDS.RADEC2xyz(r_station_earth,RA_station,DEC_station))

station_earth

[array([-0.81183633, -0.24592818,  0.52786921]),
 array([-0.81000791, -0.25188507,  0.52786921]),
 array([-0.80811365, -0.25789773,  0.52786921]),
 array([-0.80653683, -0.26278746,  0.52786921]),
 array([-0.80232652, -0.27537441,  0.52786921]),
 array([-0.80022302, -0.28142853,  0.52786921])]

In [18]:
position_fram

,UT1_hours,alpha,delta,year,month,days,deltaT
0,21.575128,142.935000,8.521111,1965,1,14,0.000000
1,21.603111,157.274167,-2.395000,1965,1,14,0.124857
2,21.631420,171.817917,-14.508333,1965,1,14,0.251173
3,21.654491,183.134167,-23.563333,1965,1,14,0.354119
4,21.714094,208.640417,-40.020278,1965,1,14,0.620067
5,21.742876,219.102083,-44.998611,1965,1,14,0.748489


In [19]:
# add Pj ,Qj to positon_fram_fil
Lt = []; Pt = []; Qt = []
Lambda = []; Miu = []; Niu = [];

#由 delta,alpha 算出 L = [Lambda,Miu,Niu]
for j in range(len(position_fram)):
    delta = position_fram['delta'].values[j]*2*np.pi/360 ; 
    alpha = position_fram['alpha'].values[j]*2*np.pi/360;

    L = [np.cos(delta)*np.cos(alpha),\
         np.cos(delta)*np.sin(alpha),\
         np.sin(delta)]
    
    Lambda.append(L[0]); 
    Miu.append(L[1]);
    Niu.append(L[2]);
    
#     Lambda.append(np.cos(delta)*np.cos(alpha))
#     Miu.append(np.cos(delta)*np.sin(alpha))
#     Niu.append(np.sin(delta))

# 由[Lambda,Miu,Niu] 和 测站-地心矢量归算出 P,Q
    Pj = L[2]*station_earth[j][0] - L[0]*station_earth[j][2]
    Qj = L[2]*station_earth[j][1] - L[1]*station_earth[j][2]
    
    Lt.append(L)
    Pt.append(Pj)
    Qt.append(Qj)
#     print(Lt,Pt,Qt)

# position_fram_fil = pd.DataFrame({'alpha':\
#                         np.array([142.935,157.274166667,171.817916667,183.134166667,\
#                                   208.640416667,219.102083333]),\
#                        'delta':\
#                        np.array([8.521111111,-2.395,-14.508333333,-23.56333333,-40.020277777,-44.998611111]),\
#                        'jd':jd_array,\
#                         'SG_d': SG_degree,\
#                         'deltaT':delta_t,\
#                         'P':np.array(Pt),'Q':np.array(Qt),\
#                         'Lambda':np.array(Lambda),\
#                         'Miu':np.array(Miu),'Niu':np.array(Niu)})

position_fram_New = pd.DataFrame({'jd':jd_array,
                                 'SG_d':SG_degree,
                                 'P':np.array(Pt),'Q':np.array(Qt),
                                  'deltaT_SI':delta_t_SI,
                                 'Lambda':np.array(Lambda),
                                 'Miu':np.array(Miu),'Niu':np.array(Niu)})

Lt 

[[-0.7891439307993768, 0.5960674272555554, 0.14817381228628504],
 [-0.9215582970478929, 0.38598449379160416, -0.04178846365477428],
 [-0.9582566000732619, 0.13778102486286736, -0.25052081271577153],
 [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885],
 [-0.6721154419353875, -0.36706452095992986, -0.6430586832976101],
 [-0.5487447629378402, -0.44598590319844417, -0.7070896402130082]]

In [20]:
position_fram_fil = pd.concat([position_fram,position_fram_New],axis=1)

In [21]:
position_fram_fil.to_csv('echo1st_format.csv')
position_fram_fil

,UT1_hours,alpha,delta,year,month,days,deltaT,Lambda,Miu,Niu,P,Q,SG_d,deltaT_SI,jd
0,21.575128,142.935000,8.521111,1965,1,14,0.000000,-0.789144,0.596067,0.148174,0.296272,-0.351086,78.032128,0.000000,-12769.600293
1,21.603111,157.274167,-2.395000,1965,1,14,0.124857,-0.921558,0.385984,-0.041788,0.520311,-0.193223,78.453011,100.735981,-12769.599128
2,21.631420,171.817917,-14.508333,1965,1,14,0.251173,-0.958257,0.137781,-0.250521,0.708283,-0.008122,78.878811,202.649000,-12769.597948
3,21.654491,183.134167,-23.563333,1965,1,14,0.354119,-0.915248,-0.050115,-0.399763,0.805554,0.131507,79.225833,285.706998,-12769.596987
4,21.714094,208.640417,-40.020278,1965,1,14,0.620067,-0.672115,-0.367065,-0.643059,0.870732,0.370844,80.122323,500.276999,-12769.594503
5,21.742876,219.102083,-44.998611,1965,1,14,0.748489,-0.548745,-0.445986,-0.707090,0.855495,0.434417,80.555226,603.889999,-12769.593304


## PART 2 给出t0,算出t0的F,G,进而得出r0,v0

In [22]:
t0 = 0.5*(position_fram_fil['deltaT'].values[0] + position_fram_fil['deltaT'].values[5])
r0_a,v0_a = Calculatefort0(position_fram_fil,[1,3,5],t0)

0 : 0.9984226162730335 0.04800349905540134
[0.03194931 0.0002037  0.07076899] [2.70966570e-03 1.36737629e-06 9.03982771e-03]
--------------------
1 : 1.5139389870973732 1.7156830515699655
[0.01971213 0.00014397 0.05538079] [1.53640244e-03 9.62029085e-07 7.26806261e-03]
--------------------
2 : 1.1180455769847804 0.5119898679876053
[1.30097450e-02 8.65400549e-05 2.84405528e-02] [1.06962099e-03 5.80545151e-07 3.45320468e-03]
--------------------
3 : 1.3277824392203847 1.189450828058739
[8.27294713e-03 5.81603520e-05 2.02816459e-02] [6.57554048e-04 3.89386192e-07 2.52542070e-03]
--------------------
4 : 1.1803973151079028 0.7278185547910484
[5.40724101e-03 3.65849407e-05 1.20782509e-02] [4.39650820e-04 2.45306250e-07 1.46361747e-03]
--------------------
5 : 1.269370389075524 1.0128768484239639
[3.47267799e-03 2.40544923e-05 8.16717117e-03] [2.78365795e-04 1.61147940e-07 1.00259023e-03]
--------------------
6 : 1.2096396785263392 0.8240466312977627
[2.25552476e-03 1.53805417e-05 5.11341846

In [28]:
r0 = np.sqrt(np.dot(r0_a,r0_a))
v0 = np.sqrt(np.dot(v0_a,v0_a))

print(r0_a*R_earth,'r0:',r0*R_earth,'km;','h:',(r0-r_station_earth)*R_earth,'km')

print(v0_a*R_earth/time_unit,'v0:',v0*R_earth/time_unit,'km/s')

print(t0*time_unit,'s')

[-7245.98540944 -1874.17826161  2365.63579833] r0: 7849.400068826888 km; h: 1484.1212268268876 km
[-0.05590008 -5.20070644 -4.79699883] v0: 7.07542720284154 km/s
301.94499960000485 s


In [298]:
r0_a*R_earth

array([-7245.98540944, -1874.17826161,  2365.63579833])

In [299]:
v0_a*R_earth/time_unit

array([-0.05590008, -5.20070644, -4.79699883])

## 2. 已知某一时刻的位置矢量和速度矢量，求轨道六根数

按照笔记方法,逐步求得：
a
n()
E()
e
M
i Omega w

In [300]:
# 从第二步开始，因为不需要用到F,G的计算，所以不用再取理论单位，现在取国际单位制
# astronomy constants
R_earth = 6371e3 # m

M_earth = 5.965e24 # kg
G_graviation = 6.672e-11 # N·m^2 /kg^2 

au = 149597870e3 # m
r_station_earth = 0.999102 #*R_earth
r_earth_sun = 1*au

time_unit = 806.81163 #806.8116 # SI 

# miu_GM = 398600.5e-6 # km^3/SI^2

In [301]:
miu_GM = G_graviation*M_earth # 国际单位制

miu_GM

397984800000000.0

In [302]:
# input r0,v0,t0
r0 = r0_a*R_earth # unit= m
r0_norm = np.sqrt(sum([ i*i for i in r0])) # m

v0 = v0_a*R_earth/time_unit # unit= m/SI
v0_norm = np.sqrt(sum([ i*i for i in v0])) # m/SI

t0 = t0*time_unit # unit = SI from the first data we have
## t0 根据自己的单位去选择。

r0_norm/10**3,v0_norm/10**3

(7849.400068826888, 7.07542720284154)

In [303]:
# get a 

a = 1/(2/r0_norm - v0_norm**2/miu_GM) # unit = m

a/R_earth

1.2166721606885087

In [304]:
# get n\E then a
n = np.sqrt(miu_GM/(a**3))

tan_E = (1 - r0_norm/a)*(a**2*n)/(np.dot(r0,v0))

e = np.sqrt( (1 - r0_norm/a)**2 + (np.dot(r0,v0) / (n*a**2))**2 )
# e = 0.01002

print(1 - r0_norm/a)
print(np.dot(r0,v0) / (n*a**2))

cos_E = (1 - r0_norm/a)/e
sin_E = (np.dot(r0,v0) / (a**2*n))/e

e

-0.012640490918745195
-0.02153043882356097


0.02496681410596837

In [305]:
def arctan2(sinE,cosE):
    if sinE >=0 and cosE >=0: # I
        output = np.arcsin(sinE)
    elif sinE >=0 and cosE <=0: # II
        output = np.pi - np.arcsin(sinE)
    elif sinE <=0 and cosE >=0: # IV
        output = 2*np.pi + np.arcsin(sinE)
    else: # III
        output = np.pi - np.arcsin(sinE)
    return output


In [306]:
print(sin_E,cos_E,tan_E)

E = arctan2(sin_E,cos_E)
print(E)
np.degrees(E)

-0.862362283476692 -0.5062917064666036 0.5870986198810116
4.181509802684073


239.58286368637903

In [307]:
# keplar 定 M
M = E - e*sin_E
M

4.203040241507634

In [308]:
# 定 i Omega w

#### method 1
P = (cos_E/r0_norm)*r0 - (sin_E/(a*n))*v0
Q = (sin_E/(r0_norm)*np.sqrt(1-e**2))*r0 + ((cos_E - e)/(a*n*np.sqrt(1-e**2)))*v0
R = np.cross(P,Q)

cos_i = R[2]
tan_Omega = -1*R[0]/R[1]

# get w ->[-90,+90]
tan_w = P[2]/Q[2]

############################################################

#### method 2
h = np.cross(r0,v0)
h_norm = np.sqrt(np.dot(h,h))

h_A, h_B, h_C = h[0:3]

# get cos_i, sin_i
cos_i_check = (h_C/h_norm)
tan_i = np.sqrt(h_A**2 + h_B**2) / h_C

sin_i = tan_i*cos_i_check


# get cos_\sin_Omega
sin_Omega = h_A/(h_norm*sin_i)
cos_Omega = -h_B/(h_norm*sin_i)

# finally get i Omega w

i = arctan2(sin_i,cos_i_check)

Omega = arctan2(sin_Omega,cos_Omega)

w = np.arctan(tan_w)

np.degrees(np.array([i,Omega,w]))

array([ 47.40567751,  31.39487317, -82.51085278])

## 3. 由轨道6根数归算星历表

已知：
* a, # m
* n(), # rad/s
* E(), # rad
* e, # Nan
* M, # rad
* [i, Omega, w] = P,Q # [rad,rad,rad]

t0 = 121.0217445 **# unit = SI from the first data we have**

M = M0 + n(t-t0)

M = E - esin(E)

In [309]:
print('a',a,'n',n,'E',E,'e',e,'M',M,i,Omega,w)

a 7751418.3357464885 n 0.0009244032824160432 E 4.181509802684073 e 0.02496681410596837 M 4.203040241507634 0.827385156755234 0.5479439050679501 -1.4400860495680883


In [310]:
# 假设我们已知a,e,M,i,Omega,w,那么E,n均需要从六根数推出来。
E_old = E
n_old = n

n = np.sqrt(miu_GM/a**3)

n

0.0009244032824160432

In [311]:
# calculate keplar equation
# 设置初值t0 和我们需要归算的时间t

# 我将计算以整个周期为倍数的时间序列
T = 2*np.pi/n * 10
print(T)

t = np.arange(-201,T,100) # unit = SI
t_forecast = t
Mt = (M + n*(t))%(2*np.pi)

print(t)
print(Mt)

67970.17521138283
[-2.0100e+02 -1.0100e+02 -1.0000e+00  9.9000e+01  1.9900e+02  2.9900e+02
  3.9900e+02  4.9900e+02  5.9900e+02  6.9900e+02  7.9900e+02  8.9900e+02
  9.9900e+02  1.0990e+03  1.1990e+03  1.2990e+03  1.3990e+03  1.4990e+03
  1.5990e+03  1.6990e+03  1.7990e+03  1.8990e+03  1.9990e+03  2.0990e+03
  2.1990e+03  2.2990e+03  2.3990e+03  2.4990e+03  2.5990e+03  2.6990e+03
  2.7990e+03  2.8990e+03  2.9990e+03  3.0990e+03  3.1990e+03  3.2990e+03
  3.3990e+03  3.4990e+03  3.5990e+03  3.6990e+03  3.7990e+03  3.8990e+03
  3.9990e+03  4.0990e+03  4.1990e+03  4.2990e+03  4.3990e+03  4.4990e+03
  4.5990e+03  4.6990e+03  4.7990e+03  4.8990e+03  4.9990e+03  5.0990e+03
  5.1990e+03  5.2990e+03  5.3990e+03  5.4990e+03  5.5990e+03  5.6990e+03
  5.7990e+03  5.8990e+03  5.9990e+03  6.0990e+03  6.1990e+03  6.2990e+03
  6.3990e+03  6.4990e+03  6.5990e+03  6.6990e+03  6.7990e+03  6.8990e+03
  6.9990e+03  7.0990e+03  7.1990e+03  7.2990e+03  7.3990e+03  7.4990e+03
  7.5990e+03  7.6990e+03  7.7990e

In [312]:
# 迭代 开普勒方程解 E
Et = []
epsilon = 1e-6
for M_single in Mt:
    print(M_single)
    key = 0
    En0 = M_single
    Enw = [En0]
    Nwind = 0
    while key == 0:
        Enw.append(M_single + e*np.sin(Enw[Nwind]))

        Nwind += 1
#         print(Enw[Nwind])

        delta_Enw = np.abs(Enw[Nwind] - Enw[Nwind - 1])
#         print(delta_Enw)
#         print('------------')

        if delta_Enw < epsilon:
            key = 1;

    E_single = Enw[Nwind]
    Et.append(E_single)
print('E:',Et)

4.017235181742009
4.109675509983614
4.202115838225218
4.294556166466823
4.386996494708427
4.4794368229500305
4.571877151191635
4.664317479433239
4.756757807674844
4.849198135916448
4.941638464158053
5.034078792399657
5.1265191206412615
5.218959448882865
5.311399777124469
5.403840105366074
5.496280433607678
5.588720761849283
5.681161090090887
5.7736014183324915
5.866041746574096
5.9584820748157
6.050922403057305
6.143362731298909
6.235803059540514
0.045058080602530914
0.13749840884413533
0.22993873708573975
0.32237906532734417
0.4148193935689486
0.5072597218105521
0.5997000500521565
0.692140378293761
0.7845807065353654
0.8770210347769698
0.9694613630185742
1.0619016912601786
1.154342019501783
1.2467823477433875
1.339222675984992
1.4316630042265963
1.5241033324682007
1.6165436607098052
1.7089839889514087
1.801424317193014
1.8938646454346184
1.9863049736762228
2.0787453019178272
2.1711856301594317
2.263625958401036
2.3560662866426387
2.448506614884243
2.5409469431258476
2.633387271367452


0.5320728410561202
0.6245131692977282
0.716953497539329
0.809393825780937
0.9018341540225379
0.9942744822641458
1.0867148105057467
1.1791551387473476
1.2715954669889555
1.3640357952305635
1.4564761234721573
1.5489164517137652
1.6413567799553732
1.7337971081969812
1.826237436438575
1.918677764680183
2.011118092921791
2.103558421163399
2.1959987494049926
2.2884390776466006
2.3808794058882086
2.4733197341298165
2.5657600623714103
2.6582003906130183
2.7506407188546262
2.843081047096234
2.935521375337828
3.027961703579436
3.120402031821044
3.212842360062652
3.3052826883042457
3.3977230165458536
3.4901633447874616
3.5826036730290696
3.6750440012706633
3.7674843295122713
3.8599246577538793
3.952364985995473
4.044805314237081
4.137245642478689
E: [3.9983669120797702, 4.08939902362585, 4.180596947853597, 4.27197183816724, 4.363533567412049, 4.455290615582357, 4.547249996092019, 4.639417108531271, 4.731795694933859, 4.824387747202183, 4.917193440582445, 5.010211071475066, 5.1034370493208625, 5.1

In [313]:
np.cos(i)

0.6768030255350356

In [314]:
# 给出P_new, Q_new
P_new = np.array([np.cos(Omega)*np.cos(w) - np.sin(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(Omega)*np.cos(w) + np.cos(Omega)*np.sin(w)*np.cos(i),\
                 np.sin(w)*np.sin(i) ])
Q_new = np.array([-np.cos(Omega)*np.sin(w) - np.sin(Omega)*np.cos(w)*np.cos(i),\
                 -np.sin(Omega)*np.sin(w) + np.cos(Omega)*np.cos(w)*np.cos(i),\
                 np.cos(w)*np.sin(i)])

print(P,Q)
print(P_new,Q_new)

[ 0.46064345 -0.50502068 -0.72990528] [0.79996673 0.59154962 0.0959532 ]
[ 0.46081815 -0.50489153 -0.72988436] [0.80036256 0.59178821 0.09595045]


In [315]:
# 归算rt, vt
rt = [];vt=[];
for Et_single in Et:
    rt_single = a*(np.cos(Et_single) - e)*P_new + a*np.sqrt(1 - e**2)*np.sin(Et_single)*Q_new
    rt_single_norm = np.sqrt( np.dot(rt_single,rt_single) )
    rt.append(rt_single)
    
    vt_single = -a**2*n/rt_single_norm*np.sin(Et_single)*P_new + a**2*n/rt_single_norm*np.sqrt(1 - e**2)*np.cos(Et_single)*Q_new
    vt_single_norm = np.sqrt( np.dot(vt_single,vt_single) )
    vt.append(rt_single)
    
#     print(rt_single_norm/10**3,vt_single_norm/10**3)
    
rt = np.array(rt)
vt = np.array(vt)

In [316]:
# 计算预测的方向矢量L，赤经，赤纬
P_earth = rt
station_earth = np.array([-0.81000791, -0.25188507,  0.52786921])*R_earth

P_station = P_earth - station_earth

L_forecast_total = [];
Alpha_forecast = [];
Delta_forecast = [];
Nwind = 0
for P_station_single in P_station:
    # 计算我们测站的赤经Alpha_forecast 和赤纬Delta_forecast
    L_forecast_total.append( P_station_single/np.sqrt(np.dot(P_station_single,P_station_single)) )
    
    Alpha_forecast.append( np.arcsin(L_forecast_total[Nwind][2]) )
    Delta_forecast.append( arctan2(L_forecast_total[Nwind][1]/np.cos(Alpha_forecast[Nwind]),\
                                   L_forecast_total[Nwind][0]/np.cos(Alpha_forecast[Nwind])) )
    
    Nwind += 1;

##############################################

Forecast_fram = pd.DataFrame({'Alpha': np.array(Alpha_forecast),
                             'Delta':np.array(Delta_forecast),
                             'deltaT':t_forecast-t_forecast[0],
                             'x':rt[:,0],
                             'y':rt[:,1],
                             'z':rt[:,2],
                             'vx':vt[:,0],
                             'vy':vt[:,1],
                             'vz':vt[:,2]})

# print(L_forecast_total)

In [317]:
Forecast_fram.to_csv('echo1st_forecast.csv')
Forecast_fram


,Alpha,Delta,deltaT,vx,vy,vz,x,y,z
0,-0.037213,2.753247,0.0,-7.115475e+06,-8.049604e+05,3.284417e+06,-7.115475e+06,-8.049604e+05,3.284417e+06
1,-0.248634,3.014617,100.0,-7.210205e+06,-1.343096e+06,2.838453e+06,-7.210205e+06,-1.343096e+06,2.838453e+06
2,-0.441586,3.268217,200.0,-7.245975e+06,-1.870245e+06,2.369282e+06,-7.245975e+06,-1.870245e+06,2.369282e+06
3,-0.592261,3.502128,300.0,-7.222160e+06,-2.382011e+06,1.880631e+06,-7.222160e+06,-2.382011e+06,1.880631e+06
4,-0.701745,3.712111,400.0,-7.138599e+06,-2.874067e+06,1.376427e+06,-7.138599e+06,-2.874067e+06,1.376427e+06
5,-0.780470,3.899672,500.0,-6.995614e+06,-3.342193e+06,8.607693e+05,-6.995614e+06,-3.342193e+06,8.607693e+05
6,-0.838076,4.068802,600.0,-6.794015e+06,-3.782312e+06,3.379042e+05,-6.794015e+06,-3.782312e+06,3.379042e+05
7,-0.881173,4.223794,700.0,-6.535111e+06,-4.190522e+06,-1.878061e+05,-6.535111e+06,-4.190522e+06,-1.878061e+05
8,-0.913868,4.368291,800.0,-6.220708e+06,-4.563140e+06,-7.119163e+05,-6.220708e+06,-4.563140e+06,-7.119163e+05
9,-0.938637,4.505037,900.0,-5.853112e+06,-4.896732e+06,-1.229933e+06,-5.853112e+06,-4.896732e+06,-1.229933e+06


In [318]:
L = [-0.9152477093203553, -0.0501154444748697, -0.39976252051552885]
L = [-0.921558,0.385984,-0.041788]
L

[-0.921558, 0.385984, -0.041788]